In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import json

In [15]:
with open('./benchmark_data/East_Massachusetts_net_Apr_PM_ext.txt') as MA_flow:
    MA_flow_lines = MA_flow.readlines()
MA_links = []
i = -8
for line in MA_flow_lines:
    i += 1
    if i > 0:
        MA_links.append(line.split('  ')[1:3])
numLinks = i

In [16]:
numLinks

74

In [17]:
link_list_js = [str(int(MA_links[i][0])) + ',' + str(int(MA_links[i][1])) for \
                i in range(len(MA_links))]

link_list_pk = [str(int(MA_links[i][0])) + '->' + str(int(MA_links[i][1])) for \
                i in range(len(MA_links))]

In [18]:
numNodes = max([int(MA_links[i][1]) for i in range(numLinks)])

In [19]:
numNodes

22

In [20]:
from collections import defaultdict

node_neighbors_dict = defaultdict(list)

for node in range(numNodes):
    for link in MA_links:
        if node == int(link[0]):
            node_neighbors_dict[str(node)].append(int(link[1]))

In [21]:
with open('./benchmark_data/East_Massachusetts_trips_Apr_PM_ext.txt') as MA_trips:
    MA_trips_lines = MA_trips.readlines()

In [22]:
numZones = int(MA_trips_lines[0].split(' ')[3])

In [23]:
numZones

22

In [24]:
od_pairs = []
for i in range(numZones+1)[1:]:
    for j in range(numZones+1)[1:]:
        if i != j:
            od_pairs.append([i, j])
            
numODpairs = len(od_pairs)

In [25]:
# create O-D pair labels
# create a dictionary mapping O-D pairs to labels

import json

OD_pair_label_dict = {}
OD_pair_label_dict_ = {}

label = 1
for i in range(numZones + 1)[1:]:
    for j in range(numZones + 1)[1:]:
        key = (i, j)
        OD_pair_label_dict[str(key)] = label
        OD_pair_label_dict_[str(label)] = key
        label += 1
        
with open('od_pair_label_dict_MA_ext.json', 'w') as json_file:
    json.dump(OD_pair_label_dict, json_file)
    
with open('od_pair_label_dict__MA_ext.json', 'w') as json_file:
    json.dump(OD_pair_label_dict_, json_file)


OD_pair_label_dict_refined = {}
OD_pair_label_dict_refined_ = {}

label = 1
for i in range(numZones + 1)[1:]:
    for j in range(numZones + 1)[1:]:
        if i != j:
            key = (i, j)
            OD_pair_label_dict_refined[str(key)] = label
            OD_pair_label_dict_refined_[str(label)] = key
            label += 1
        
with open('od_pair_label_dict_MA_refined_ext.json', 'w') as json_file:
    json.dump(OD_pair_label_dict_refined, json_file)
    
with open('od_pair_label_dict__MA_refined_ext.json', 'w') as json_file:
    json.dump(OD_pair_label_dict_refined_, json_file)
    
    
# create link labels
# create a dictionary mapping directed links to labels
link_label_dict = {}
link_label_dict_ = {}

for i in range(numLinks):
    link_label_dict[str(i)] = link_list_js[i]

for i in range(numLinks):
    link_label_dict_[link_list_js[i]] = i

with open('link_label_dict_MA_ext.json', 'w') as json_file:
    json.dump(link_label_dict, json_file)
    
with open('link_label_dict_MA__ext.json', 'w') as json_file:
    json.dump(link_label_dict_, json_file)
    
# create link labels
# create a dictionary mapping directed links to labels
link_label_dict = {}
link_label_dict_ = {}

for i in range(numLinks):
    link_label_dict[str(i)] = link_list_pk[i]

for i in range(numLinks):
    link_label_dict_[link_list_pk[i]] = i

zdump(link_label_dict, 'link_label_dict_MA_network_ext.pkz')
zdump(link_label_dict_, 'link_label_dict_MA_network__ext.pkz')

In [28]:
link_length_list = []
with open('./benchmark_data/East_Massachusetts_net_Apr_PM_ext.txt', 'r') as f:
    read_data = f.readlines()
    flag = 0
    for row in read_data:
        if ';' in row:
            flag += 1
            if flag > 1:
                link_length_list.append(float(row.split('  ')[5]))

In [31]:
link_length_list

[0.161995,
 0.183317,
 0.181959,
 0.179745,
 0.34941,
 0.348965,
 0.109833,
 0.107958,
 0.117963,
 0.12994,
 0.282064,
 0.284951,
 0.364935,
 0.362124,
 0.06363,
 0.095444,
 0.071077,
 0.106616,
 0.043136,
 0.045919,
 0.173468,
 0.163501,
 0.177869,
 0.19188,
 0.325329,
 0.332162,
 0.068599,
 0.068377,
 0.09917,
 0.103701,
 0.101832,
 0.102047,
 0.291648,
 0.289254,
 0.211194,
 0.202784,
 0.209099,
 0.212578,
 0.06683,
 0.066264,
 0.11861,
 0.119511,
 0.467869,
 0.474671,
 0.069387,
 0.104081,
 0.069605,
 0.104407,
 0.091506,
 0.137259,
 0.096101,
 0.144152,
 0.157933,
 0.15808,
 0.104414,
 0.156622,
 0.103075,
 0.154612,
 0.131017,
 0.130002,
 0.414111,
 0.465782,
 0.158994,
 0.169855,
 0.376906,
 0.384494,
 0.079154,
 0.118731,
 0.079134,
 0.118701,
 0.245197,
 0.243392,
 0.194284,
 0.208304]

In [32]:
link_label_dict = zload('link_label_dict_MA_network_ext.pkz')
link_label_dict_ = zload('link_label_dict_MA_network__ext.pkz')

In [33]:
import networkx as nx

def jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, link_list_js, link_length_list):
    MA = nx.DiGraph()

    MA.add_nodes_from(range(numNodes+1)[1:])

    MA_weighted_edges = [(int(link_list_js[i].split(',')[0]), int(link_list_js[i].split(',')[1]), \
                             link_length_list[i]) for i in range(len(link_list_js))]

    MA.add_weighted_edges_from(MA_weighted_edges)

    path = nx.all_pairs_dijkstra_path(MA)

    od_route_dict = {}
    for od in od_pairs:
        origi = od[0]
        desti = od[1]
        key = OD_pair_label_dict_refined[str((origi, desti))]
        route = str(path[origi][desti]).replace("[", "").replace(", ", "->").replace("]", "")
        od_route_dict[key] = route

    od_link_dict = {}
    for idx in range(len(od_route_dict)):
        od_link_list = []
        od_node_list = od_route_dict[idx+1].split('->')
        for i in range(len(od_node_list)):
            if i < len(od_node_list) - 1:
                od_link_list.append(link_label_dict_[od_node_list[i] + '->' + od_node_list[i+1]])
        od_link_dict[idx] = od_link_list

    jacob = np.zeros((numODpairs, numLinks))

    for i in range(numODpairs):
        for j in range(numLinks):
            if j in od_link_dict[i]:
                jacob[i, j] = 1

    return jacob

In [34]:
jacob = jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, link_list_js, link_length_list)

In [36]:
jacob.shape

(462, 74)

In [37]:
jacob

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])